In [1]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Data Preparation

In [3]:
from Scripts.Data.NFL_data_preprocessing import preprocess_nfl, open_all_tsv

dataframes = open_all_tsv('Data/NFL/')

dataframes = preprocess_nfl(dataframes)

Conversion completed for all TSV files in the folder.


In [4]:
def prepare_data(dataframes, test_games):
    X_train, Y_train, Y_train_noised = {}, {}, {}
    X_test, Y_test, Y_test_noised = {}, {}, {}
    num_success = 0
    num_fail = 0

    # Noise process parameters
    N = 1000
    gamma_min = 0.1
    gamma_max = 0.9
    gammas = np.linspace(gamma_max, gamma_min, N)

    for df_name, df in dataframes.items():
        # Skip the test game in the training set creation
        if df_name in test_games:
            continue

        # Reset the index of the DataFrame if 'playerId' is an index
        if 'playerId' in df.index.names:
            df.reset_index('playerId', drop=True, inplace=True)

        # Keep only numeric columns in the dataframe
        df = df.select_dtypes(include=[np.number])

        # Apply the noise process to the dataframe
        noisy_df = np.copy(df.values)
        for t in range(1, len(df)):
            k = min(t, N-1)  # Ensure not to exceed the number of defined gammas
            gamma = gammas[k]
            Z_t = np.random.normal(0, 1, size=df.iloc[t].shape)
            noisy_df[t] = gamma * df.iloc[t - 1] + np.sqrt(1 - gamma**2) * Z_t

        # Calculate the difference between the original and noisy trajectories
        y = df.values - noisy_df

        try:
            # Reshape the data to the format (number_frames, number_players, number_features)
            num_frames = int(df.shape[0] / 23)
            num_features = df.shape[1]
            x = df.values.reshape(num_frames, 23, num_features)
            y = y.reshape(num_frames, 23, num_features)

            # Remove one frame if the number of frames is odd
            if num_frames % 2 != 0:
                x = x[:-1]  # Remove the last frame
                y = y[:-1]  # Remove the last frame

            # Split the data into two equal parts
            split_index = int(num_frames * 0.5)

            x, y_target = x[:split_index], x[split_index:]
            y_noised, y_target_noised = y[:split_index], y[split_index:]

            # Append to respective dictionaries
            X_train[df_name] = torch.tensor(x, dtype=torch.float32)
            Y_train[df_name] = torch.tensor(y_target, dtype=torch.float32)
            Y_train_noised[df_name] = torch.tensor(y_target_noised, dtype=torch.float32)
            num_success += 1
        except ValueError:
            print(f"Failed to reshape dataframe '{df_name}'")
            num_fail += 1

    print(f"Number of successfully converted dataframes: {num_success}")
    print(f"Number of failed conversions: {num_fail}")

    # Prepare the test set
    for test_game in test_games:
        test_df = dataframes[test_game]

        if 'playerId' in test_df.index.names:
            test_df.reset_index('playerId', drop=True, inplace=True)
        test_df = test_df.select_dtypes(include=[np.number])
        noisy_df = np.copy(test_df.values)
        for t in range(1, len(test_df)):
            k = min(t, N-1)
            gamma = gammas[k]
            Z_t = np.random.normal(0, 1, size=test_df.iloc[t].shape)
            noisy_df[t] = gamma * test_df.iloc[t - 1] + np.sqrt(1 - gamma**2) * Z_t

        y = test_df.values - noisy_df
        num_frames = int(test_df.shape[0] / 23)
        num_features = test_df.shape[1]
        x = test_df.values.reshape(num_frames, 23, num_features)
        y = y.reshape(num_frames, 23, num_features)

        # Remove one frame if the number of frames is odd
        if num_frames % 2 != 0:
            x = x[:-1]  # Remove the last frame
            y = y[:-1]  # Remove the last frame

        # Split the data into two equal parts
        split_index = int(num_frames * 0.5)

        x, y_target = x[:split_index], x[split_index:]
        y_noised, y_target_noised = y[:split_index], y[split_index:]

        # Append to respective dictionaries
        X_test[test_game] = torch.tensor(x, dtype=torch.float32)
        Y_test[test_game] = torch.tensor(y_target, dtype=torch.float32)
        Y_test_noised[test_game] = torch.tensor(y_target_noised, dtype=torch.float32)

    return X_train, Y_train, Y_train_noised, X_test, Y_test, Y_test_noised



test_games = ['2019_WAS_2019090806_193', '2019_WAS_2019090806_1117', '2019_WAS_2019092907_2823', '2019_WAS_2019100609_431', '2019_WAS_2019111707_3624', '2019_WAS_2019120803_3729', '2019_WAS_2019121508_700']

# Assume `dataframes` is your preprocessed data
X_train, Y_train, Y_train_noised, X_test, Y_test, Y_test_noised = prepare_data(dataframes, test_games)

Number of successfully converted dataframes: 554
Number of failed conversions: 0


In [5]:
import pickle

# Save
with open('Data/nfl_train_test_data_diff.pkl', 'wb') as f:
    pickle.dump({'X_train': X_train, 'Y_train': Y_train, 'Y_train_noised':Y_train_noised, 'X_test': X_test, 'Y_test': Y_test, 'Y_test_noised':Y_test_noised}, f)

## Training Model

In [6]:
import pickle 

# Load
with open('Data/nfl_train_test_data_diff.pkl', 'rb') as f:
    data = pickle.load(f)
    X_train = data['X_train']
    Y_train = data['Y_train']
    Y_train_noised = data['Y_train_noised']
    X_test = data['X_test']
    Y_test = data['Y_test']
    Y_test_noised = data['Y_test_noised']

In [7]:
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Initialize the dictionaries for the normalized data
X_train_normalized = {}
Y_train_normalized = {}
Y_train_noised_normalized = {}
X_test_normalized = {}
Y_test_normalized = {}
Y_test_noised_normalized = {}

# Initialize the scalers
scaler_x = StandardScaler()
scaler_y = StandardScaler()
scaler_y_noised = StandardScaler()

# Normalize the training data
for game, x in X_train.items():
    y = Y_train[game]
    y_noised = Y_train_noised[game]

    # Flatten the tensors along the frame and player dimensions
    x_flattened = x.view(x.shape[0] * x.shape[1], -1).numpy()
    y_flattened = y.view(y.shape[0] * y.shape[1], -1).numpy()
    y_noised_flattened = y_noised.view(y_noised.shape[0] * y_noised.shape[1], -1).numpy()

    # Fit the scalers to the flattened tensors and transform
    x_normalized_np = scaler_x.fit_transform(x_flattened)
    y_normalized_np = scaler_y.fit_transform(y_flattened)
    y_noised_normalized_np = scaler_y_noised.fit_transform(y_noised_flattened)

    # Convert the normalized arrays back to tensors and reshape to original shape
    x_normalized = torch.tensor(x_normalized_np, dtype=torch.float32).view(x.shape)
    y_normalized = torch.tensor(y_normalized_np, dtype=torch.float32).view(y.shape)
    y_noised_normalized = torch.tensor(y_noised_normalized_np, dtype=torch.float32).view(y_noised.shape)

    # Append to the dictionaries
    X_train_normalized[game] = x_normalized
    Y_train_normalized[game] = y_normalized
    Y_train_noised_normalized[game] = y_noised_normalized

# Normalize the testing data
for game, x in X_test.items():
    y = Y_test[game]
    y_noised = Y_test_noised[game]

    # Flatten the tensors along the frame and player dimensions
    x_flattened = x.view(x.shape[0] * x.shape[1], -1).numpy()
    y_flattened = y.view(y.shape[0] * y.shape[1], -1).numpy()
    y_noised_flattened = y_noised.view(y_noised.shape[0] * y_noised.shape[1], -1).numpy()

    # Transform the flattened tensors using the fitted scalers
    x_normalized_np = scaler_x.fit_transform(x_flattened)
    y_normalized_np = scaler_y.fit_transform(y_flattened)
    y_noised_normalized_np = scaler_y_noised.fit_transform(y_noised_flattened)

    # Convert the normalized arrays back to tensors and reshape to original shape
    x_normalized = torch.tensor(x_normalized_np, dtype=torch.float32).view(x.shape)
    y_normalized = torch.tensor(y_normalized_np, dtype=torch.float32).view(y.shape)
    y_noised_normalized = torch.tensor(y_noised_normalized_np, dtype=torch.float32).view(y_noised.shape)

    # Append to the dictionaries
    X_test_normalized[game] = x_normalized
    Y_test_normalized[game] = y_normalized
    Y_test_noised_normalized[game] = y_noised_normalized

In [8]:
from Scripts.Results.Metrics.Evaluation_Metrics import train_diff
from Scripts.Models.SocialTransformer4 import DiffusionTransformer

input_size = list(X_train_normalized.values())[0].shape[2]
output_size = list(Y_train_normalized.values())[0].shape[2]
hidden_size = 256
num_layers = 2
nhead = 6
dropout = 0.1
max_len = 5000
model = DiffusionTransformer(d_model=input_size, nhead=nhead, num_encoder_layers=num_layers, d_ff=hidden_size, dropout=dropout, max_len=max_len)

batch_size = 32
learning_rate = 0.001
epochs = 10

In [10]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import time
from torch.optim.lr_scheduler import ReduceLROnPlateau

from Scripts.Results.Metrics.Evaluation_Metrics import train_diff
from Scripts.Models.SocialTransformer4 import DiffusionTransformer

input_size = list(X_train_normalized.values())[0].shape[2]
output_size = list(Y_train_normalized.values())[0].shape[2]
hidden_size = 256
num_layers = 2
nhead = 6
dropout = 0.1
max_len = 5000
model = DiffusionTransformer(d_model=input_size, nhead=nhead, num_encoder_layers=num_layers, d_ff=hidden_size, dropout=dropout, max_len=max_len)

batch_size = 32
learning_rate = 0.001
epochs = 10

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=3, verbose=True)

model.train()
best_val_loss = float('inf')
no_improvement = 0
patience = 3


for epoch in range(epochs):
    start_time = time.time()
    train_loss = 0.0
    for df_name, x in X_train_normalized.items():
        y = Y_train_normalized[df_name]
        y_noised = Y_train_noised_normalized[df_name]
        for i in range(0, len(x), batch_size):
            optimizer.zero_grad()
            batch_x = x[i:i+batch_size]
            batch_y = y[i:i+batch_size]
            batch_y_noised = y_noised[i:i+batch_size]
            output = model(batch_x, batch_y_noised)
            output = batch_y+output

            loss = criterion(output, batch_y_noised)

            loss.backward()
            optimizer.step()
            train_loss += loss.item()

    train_loss /= len(X_train)

    # Evaluation on the test set
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for test_game, x in X_test_normalized.items():
            x = X_test_normalized[test_game]
            y = Y_test_normalized[test_game]
            y_noised = Y_test_noised_normalized[test_game]
            for i in range(0, len(x), batch_size):
                batch_x = x[i:i+batch_size]
                batch_y = y[i:i+batch_size]
                batch_y_noised = y_noised[i:i+batch_size]
                output = model(batch_x, batch_y_noised)
                loss = criterion(output+batch_y, batch_y_noised)

                test_loss += loss.item()

        test_loss /= len(X_test)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f},  Time: {time.time() - start_time:.4f}s")

    if test_loss < best_val_loss:
        best_val_loss = test_loss
        no_improvement = 0
        torch.save(model.state_dict(), './Visualization/Results/socialdiff_model3.pt')
    else:
        no_improvement += 1
        if no_improvement >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break

    scheduler.step(test_loss)

model.eval()

Epoch 1/10, Train Loss: 0.5046, Test Loss: 0.0698,  Time: 104.4583s
Epoch 2/10, Train Loss: 0.0869, Test Loss: 0.0385,  Time: 99.2904s
Epoch 3/10, Train Loss: 0.0612, Test Loss: 0.0339,  Time: 100.9709s
Epoch 4/10, Train Loss: 0.0543, Test Loss: 0.0311,  Time: 102.5366s
Epoch 5/10, Train Loss: 0.0510, Test Loss: 0.0308,  Time: 100.4723s
Epoch 6/10, Train Loss: 0.0492, Test Loss: 0.0305,  Time: 101.0724s
Epoch 7/10, Train Loss: 0.0479, Test Loss: 0.0303,  Time: 102.2808s
Epoch 8/10, Train Loss: 0.0472, Test Loss: 0.0303,  Time: 100.5009s
Epoch 9/10, Train Loss: 0.0466, Test Loss: 0.0303,  Time: 100.9834s
Epoch 10/10, Train Loss: 0.0462, Test Loss: 0.0303,  Time: 102.4550s


DiffusionTransformer(
  (pos_encoding): RotaryPositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (social_encoder): SocialEncoder(
    (linear_q): Linear(in_features=12, out_features=12, bias=True)
    (linear_k): Linear(in_features=12, out_features=12, bias=True)
    (linear_v): Linear(in_features=12, out_features=12, bias=True)
    (linear_out): Linear(in_features=12, out_features=12, bias=True)
    (attention): MultiHeadAttention(
      (linear_q): Linear(in_features=12, out_features=12, bias=True)
      (linear_k): Linear(in_features=12, out_features=12, bias=True)
      (linear_v): Linear(in_features=12, out_features=12, bias=True)
      (linear_out): Linear(in_features=12, out_features=12, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (feedforward): FeedForward(
      (linear1): Linear(in_features=12, out_features=48, bias=True)
      (linear2): Linear(in_features=48, out_features=12, bias=True)
      (dropout): Dropout(p=0.1, inplace=Fals

In [12]:
from sklearn.metrics import mean_squared_error, r2_score

# Test the model on the test set
model.load_state_dict(torch.load('./Visualization/Results/socialdiff_model3.pt'))
model.eval()
test_loss = 0.0
criterion = nn.MSELoss()
with torch.no_grad():
    for test_game, x_test in X_test_normalized.items():
        y_test = Y_test_normalized[test_game]
        y_test_noised = Y_test_noised_normalized[test_game]
        output = model(x_test, y_test_noised)
        loss = criterion(output+y_test, y_test_noised)
        test_loss += loss.item()

test_loss /= len(X_test)
print(f'Test Loss: {test_loss:.4f}')

# Analyze the predictions on the test set
predicted_noises = []
actual_noises = []
with torch.no_grad():
    for test_game, x_test in X_test_normalized.items():
        y_test = Y_test_normalized[test_game]
        y_test_noised = Y_test_noised_normalized[test_game]
        output = model(x_test, y_test_noised)
        predicted_noise = output+y_test
        predicted_noises.append(predicted_noise.detach().numpy().reshape(-1, output.shape[-1]))
        actual_noises.append(y_test_noised.numpy().reshape(-1, y_test.shape[-1]))

# Flatten the predicted noises and actual noises for analysis
predicted_noises = np.concatenate(predicted_noises)
actual_noises = np.concatenate(actual_noises)

# Calculate the mean squared error and R^2 score
mse = mean_squared_error(actual_noises, predicted_noises)
r2 = r2_score(actual_noises, predicted_noises)
print(f'Mean Squared Error: {mse:.4f}')
print(f'R^2 Score: {r2:.4f}')

Test Loss: 0.0058
Mean Squared Error: 0.0056
R^2 Score: 0.9111


In [15]:
from Scripts.Results.Metrics.Evaluation_Metrics import min_ade23, min_fde23, reverse_diffusion_process


num_samples = 1000
pred_trajs_list = []
with torch.no_grad():
    for test_game, x_test in X_test_normalized.items():
        pred_trajs = []
        y_test = Y_test_normalized[test_game]
        y_test_noised = Y_test_noised_normalized[test_game]
        output = model(x_test, y_test_noised)
        noise_predictions = output.detach().cpu().numpy()
        for _ in range(num_samples):
            x_pred = reverse_diffusion_process(model, x_test, noise_predictions)
            pred_trajs.append(x_pred.detach().cpu().numpy())
        pred_trajs_list.append(pred_trajs)

min_ade23_list = []
min_fde23_list = []
for pred_trajs, gt_traj in zip(pred_trajs_list, X_test_normalized.values()):
    gt_traj = gt_traj.detach().cpu().numpy()
    min_ade23_ = min_ade23(pred_trajs, gt_traj)
    min_fde23_ = min_fde23(pred_trajs, gt_traj)
    min_ade23_list.append(min_ade23_)
    min_fde23_list.append(min_fde23_)

avg_min_ade23 = np.mean(min_ade23_list)
avg_min_fde23 = np.mean(min_fde23_list)

print(f'minADE23: {avg_min_ade23:.4f}, minFDE23: {avg_min_fde23:.4f}')

: 